In [7]:
from orkg import ORKG, Hosts

In [8]:
orkg = ORKG(host=Hosts.PRODUCTION)

In [16]:
orkg.predicates.by_id(id='P7057')

(Success) {'id': 'P7057', 'label': 'has subProblem', 'description': None, 'created_at': '2020-01-29T17:10:44.528+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'predicate'}

In [15]:
orkg.predicates.get(q='subProblem',exact=False,size=30,sort='label',desc=True)

(Success) [{'id': 'P7057', 'label': 'has subProblem', 'description': None, 'created_at': '2020-01-29T17:10:44.528+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'predicate'}]

In [20]:
orkg.classes.by_id(id='Resource')

(Fail) b'{"status":404,"error":"Not Found","path":"/api/classes/Resource/","timestamp":"2025-06-27T12:27:56.54404209+02:00","message":"Class \\"Resource\\" not found."}'

In [21]:
orkg.classes.get_all(q='Resource', exact=False)

(Success) [{'id': 'C15002', 'label': 'Resource', 'uri': 'http://www.w3.org/ns/dcat#Resource', 'description': None, 'created_at': '2021-04-22T13:30:56.155639+02:00', 'created_by': '7adcfc45-b315-4df7-b9a5-010e8f1ca9cb', 'modifiable': True, '_class': 'class'}, {'id': 'Resources', 'label': 'Resource', 'uri': None, 'description': None, 'created_at': '2021-11-02T10:18:04.524891+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'class'}, {'id': 'C21057', 'label': 'XrayResource', 'uri': None, 'description': None, 'created_at': '2021-11-16T19:58:09.620608+01:00', 'created_by': '1bfbb658-6a8e-4a3a-86e4-2e24b021ba9b', 'modifiable': True, '_class': 'class'}, {'id': 'C58034', 'label': 'Resource Size', 'uri': 'https://w3id.org/semsys/ns/swemls#ResourceSize', 'description': None, 'created_at': '2023-03-15T14:11:20.502354Z', 'created_by': '61a04b37-e5a3-40a1-a767-81adf2ce706e', 'modifiable': True, '_class': 'class'}, {'id': 'C58035', 'label': 'Resource Type',

In [22]:
from rdflib import Graph

# Load the RDF schema
g = Graph()
g.parse("schema.rdf", format="xml")  # Or 'turtle'/'n3' depending on your file


<Graph identifier=Nb5646c1c5d6941b0a1c78fbd5a6e0eec (<class 'rdflib.graph.Graph'>)>

In [ ]:
from rdflib import RDF, RDFS, OWL
from sentence_transformers import SentenceTransformer, util

# Load schema
g = Graph()
g.parse("schema.rdf", format="xml")  # or 'ttl', depending on format

# Load transformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Extract terms with labels/comments
seen = set()
terms = []

for s in set(g.subjects()):  # ensures unique subjects
    if s in seen:
        continue
    seen.add(s)

    label = g.value(s, RDFS.label)
    comment = g.value(s, RDFS.comment)

    type_ = g.value(s, RDF.type)
    if type_ == OWL.Class or type_ == RDFS.Class:
        kind = "Class"
    elif type_ in {RDF.Property, OWL.ObjectProperty, OWL.DatatypeProperty}:
        kind = "Property"
    else:
        kind = "Term"

    if label or comment:
        text = f"{kind}: {label or ''}. Comment: {comment or ''}"
        terms.append((s, text.strip()))

# Step 2: Encode terms
term_texts = [text for _, text in terms]
term_embeddings = model.encode(term_texts, convert_to_tensor=True)

# Step 3: Encode question
question = "Who is the author of the paper?"
question_embedding = model.encode(question, convert_to_tensor=True)

# Step 4: Compute similarity
cos_scores = util.cos_sim(question_embedding, term_embeddings)[0]
top_results = cos_scores.topk(5)
for score, idx in zip(top_results.values, top_results.indices):
    uri, desc = terms[idx]



In [40]:

for score, idx in zip(top_results.values, top_results.indices):
    uri, desc = terms[idx]
    print(f"URI: {uri}")
    print(f"Description: {desc}")
    print(f"Score: {score:.4f}")
    print("---")

URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#authorOf
Description: Property: authorOf. Comment: The creator is the author of the publication.
Score: 0.5394
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#authoredBy
Description: Property: authoredBy. Comment: The publication is authored by the creator.
Score: 0.5217
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#numberOfCreators
Description: Property: numberOfCreators. Comment: The number of creators who created this publication.
Score: 0.4766
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#creatorOf
Description: Property: creatorOf. Comment: The creator of the publication.
Score: 0.4760
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#AuthorSignature
Description: Class: AuthorSignature. Comment: The information that links a publication to an author.
Score: 0.4635
---


In [19]:
from sentence_transformers import SentenceTransformer, util
import json

model = SentenceTransformer('all-MiniLM-L6-v2')

# Load your JSON file
with open('DBLP-QuAD/DBLP-QuAD/train/questions.json', 'r') as f:
    data = json.load(f)['questions']

# Index the existing questions
questions_text = [q['question']['string'] for q in data]
embeddings = model.encode(questions_text, convert_to_tensor=True)

# Encode your new question
new_question = "Who wrote papers authored by Amel Gader?"
new_embedding = model.encode(new_question, convert_to_tensor=True)

# Compute similarity
import torch
similarities = util.pytorch_cos_sim(new_embedding, embeddings)[0]
top_result = torch.argmax(similarities).item()

# Retrieve the closest match
closest_question = data[top_result]
print("Closest question:", closest_question['question']['string'])
print("SPARQL:", closest_question['query']['sparql'])


Closest question: Which publications did B. J. Geisler author?
SPARQL: SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/266/5297> }


In [20]:
data

[{'id': 'Q0001',
  'query_type': 'SINGLE_FACT',
  'question': {'string': 'What are the papers written by the person Wazir Muhammad?'},
  'paraphrased_question': {'string': 'Which papers did the author Wazir Muhammad write?'},
  'query': {'sparql': 'SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/211/3355> }'},
  'template_id': 'TC01',
  'entities': ['<https://dblp.org/pid/211/3355>'],
  'relations': ['<https://dblp.org/rdf/schema#authoredBy>'],
  'temporal': False,
  'held_out': False},
 {'id': 'Q0002',
  'query_type': 'SINGLE_FACT',
  'question': {'string': 'What is the Wikidata ID of Yvo Desmedt?'},
  'paraphrased_question': {'string': 'The author Y. Desmedt is associated with which Wikidata identifier?'},
  'query': {'sparql': 'SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/d/YvoDesmedt> <https://dblp.org/rdf/schema#wikidata> ?answer }'},
  'template_id': 'TC04',
  'entities': ['<https://dblp.org/pid/d/YvoDesmedt>'],
  'r

In [52]:
import json

questions = data

# Store one question per template_id
template_representatives = {}
for q in questions:
    template_id = q.get("template_id")
    if template_id not in template_representatives:
        template_representatives[template_id] = {
            "id": q["id"],
            "question": q["question"]["string"],
            "sparql": q["query"]["sparql"]
        }

# Example: print the results
for template_id, entry in template_representatives.items():
    print(f"Template: {template_id}")
    print(f"  Question ID: {entry['id']}")
    print(f"  Question: {entry['question']}")
    print(f"  SPARQL: {entry['sparql']}")
    print("-" * 50)

# Optional: save to a new file
with open('template_representatives.json', 'w', encoding='utf-8') as f:
    json.dump(template_representatives, f, indent=2, ensure_ascii=False)


Template: TC01
  Question ID: Q0001
  Question: What are the papers written by the person Wazir Muhammad?
  SPARQL: SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/211/3355> }
--------------------------------------------------
Template: TC04
  Question ID: Q0002
  Question: What is the Wikidata ID of Yvo Desmedt?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/d/YvoDesmedt> <https://dblp.org/rdf/schema#wikidata> ?answer }
--------------------------------------------------
Template: TC02
  Question ID: Q0003
  Question: What is the primary affiliation of Leandro Krug Wives?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/w/LeandroKrugWives> <https://dblp.org/rdf/schema#primaryAffiliation> ?answer }
--------------------------------------------------
Template: TC05
  Question ID: Q0004
  Question: What is the webpage of Ravi Kumar?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/k/RaviKu

In [54]:
import json



with open('DBLP-QuAD/DBLP-QuAD/valid/questions.json', 'r') as f:
    data = json.load(f)['questions']
questions = data

# Store one question per template_id
val_template_representatives = {}
for q in questions:
    template_id = q.get("template_id")
    if template_id not in val_template_representatives:
        val_template_representatives[template_id] = {
            "id": q["id"],
            "question": q["question"]["string"],
            "sparql": q["query"]["sparql"]
        }

# Example: print the results
for template_id, entry in val_template_representatives.items():
    print(f"Template: {template_id}")
    print(f"  Question ID: {entry['id']}")
    print(f"  Question: {entry['question']}")
    print(f"  SPARQL: {entry['sparql']}")
    print("-" * 50)

# Optional: save to a new file
with open('val_template_representatives.json', 'w', encoding='utf-8') as f:
    json.dump(val_template_representatives, f, indent=2, ensure_ascii=False)

Template: TC02
  Question ID: Q0001
  Question: What is the primary affiliation of Zhang, Yu?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/50/671-33> <https://dblp.org/rdf/schema#primaryAffiliation> ?answer }
--------------------------------------------------
Template: TC01
  Question ID: Q0003
  Question: What are the papers written by the person Wei Li?
  SPARQL: SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/64/6025-131> }
--------------------------------------------------
Template: TC03
  Question ID: Q0012
  Question: What is the ORCID of the person Tobias K.?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/68/6676> <https://dblp.org/rdf/schema#orcid> ?answer }
--------------------------------------------------
Template: TC05
  Question ID: Q0022
  Question: What is the webpage of the author Koutschan, Christoph?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/31/4524> <https

In [70]:
with open('DBLP-QuAD/DBLP-QuAD/test/questions.json', 'r') as f:
    data = json.load(f)['questions']
questions = data

# Store one question per template_id
test_template_representatives = {}
for q in questions:
    template_id = q.get("template_id")
    if template_id not in test_template_representatives:
        test_template_representatives[template_id] = {
            "id": q["id"],
            "question": q["question"]["string"],
            "sparql": q["query"]["sparql"]
        }

# Example: print the results
for template_id, entry in test_template_representatives.items():
    print(f"Template: {template_id}")
    print(f"  Question ID: {entry['id']}")
    print(f"  Question: {entry['question']}")
    print(f"  SPARQL: {entry['sparql']}")
    print("-" * 50)

# Optional: save to a new file
with open('test_template_representatives.json', 'w', encoding='utf-8') as f:
    json.dump(test_template_representatives, f, indent=2, ensure_ascii=False)

Template: TC04
  Question ID: Q0001
  Question: Show the Wikidata ID of the person Robert Schober.
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/95/2265> <https://dblp.org/rdf/schema#wikidata> ?answer }
--------------------------------------------------
Template: TC01
  Question ID: Q0002
  Question: Which papers did the author Rodr\u00EDguez, S. write?
  SPARQL: SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/04/6636> }
--------------------------------------------------
Template: TC05
  Question ID: Q0003
  Question: What is the webpage of the person Mohammad Soleymani?
  SPARQL: SELECT DISTINCT ?answer WHERE { <https://dblp.org/pid/s/MohammadSoleymani> <https://dblp.org/rdf/schema#webpage> ?answer }
--------------------------------------------------
Template: TC02
  Question ID: Q0007
  Question: The author Sameh S. Askar is primarily affiliated to which institution?
  SPARQL: SELECT DISTINCT ?answer WHERE { <ht

In [65]:
list_1=sorted(list(template_representatives.keys()))

In [66]:
list_2=sorted(list(val_template_representatives.keys()))

In [71]:
list_3=sorted(list(test_template_representatives.keys()))

In [69]:
only_in_list1 = list(set(list_1) - set(list_2))
print("In list1 but not in list2:", only_in_list1)

# Elements in list2 but not in list1
only_in_list2 = list(set(list_2) - set(list_1))
print("In list2 but not in list1:", only_in_list2)

In list1 but not in list2: ['TP71']
In list2 but not in list1: ['TC62', 'TC36', 'TP54', 'TP15', 'TC42', 'TC17', 'TC22', 'TP72', 'TP64', 'TP32', 'TP84', 'TP04', 'TC13', 'TC74', 'TP21', 'TC83', 'TC03', 'TP11', 'TC52']


In [74]:
only_in_list3 = list(set(list_3) - set(list_1))
print("In list3 but not in list1", only_in_list3)


In list3 but not in list21 ['TC62', 'TC36', 'TP54', 'TP15', 'TC42', 'TC17', 'TC22', 'TP72', 'TP64', 'TP32', 'TP84', 'TP04', 'TC13', 'TC74', 'TP21', 'TC83', 'TC03', 'TP11', 'TC52']


In [76]:
import requests


response = requests.get('https://dblp.org/search/author/api?q=Ruijie%20Wang')

In [97]:
import requests

response = requests.get('https://dblp.org/search/author/api', params={'q': 'Ruijie Wang', 'format': 'json'})

if response.status_code == 200:
    data = response.json()
    print(data['result']['hits']['hit'][0]['info']['url'])
else:
    print("Request failed with status:", response.status_code)


https://dblp.org/pid/57/5759


In [107]:
import requests

response = requests.get('https://dblp.org/search/publ/api', params={'q': 'Quelques utilisations de la struction', 'format': 'json'})

if response.status_code == 200:
    data = response.json()
    print(data['result']['hits']['hit'][0]['info']['url'])
else:
    print("Request failed with status:", response.status_code)

https://dblp.org/rec/journals/dm/Hertz86


In [17]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [1]:
import re
import codecs

import re
import codecs

def preprocess_text(text):
    # Decode unicode escape sequences like \\u00F3 → ó
    try:
        text = codecs.decode(text, 'unicode_escape')
    except Exception:
        pass  # Fail silently if already decoded

    # Remove escaped periods like '\.' → '.'
    text = text.replace('\\.', '')

    # Remove comma between two name-like words (Firstname, Lastname or Lastname, Firstname)
    # E.g., "Ramón, Sánchez" → "Ramón Sánchez"
    text = re.sub(r'(\b[\w\-\']+\b),\s*(\b[\w\-\']+\b)', r'\1 \2', text)

    return text



In [150]:
preprocess_text("What are all the papers that Tatu, Marta and Dmitry Strebkov published in SBD@SIGMOD?")

'What are all the papers that Tatu Marta and Dmitry Strebkov published in SBD@SIGMOD?'

In [18]:
#doc = nlp("Did the author Doremieux, O. publish the paper 'The PANTHER database of protein families, subfamilies, functions and pathways' in Nucleic Acids Res.?")
#doc = nlp("Have Fernando M. and Ram\\u00F3n, \\. S\\u00E1nchez not co-authored the paper 'A Model Driven Approach for Generating Code from Security Requirements'?")
#doc = nlp("Did the authors Zbikowski, K. and Ostapowicz, M. not not co-author a paper?")
#doc = nlp("What are all the papers that Tatu, Marta and Dmitry Strebkov published in SBD@SIGMOD?")

doc = nlp(preprocess_text("t the co-authors of Nianxia Cao and where are they affiliated."))
for ent in doc.ents:
    if ent.label_=='PERSON':
        print(ent.text)

Nianxia Cao


In [15]:
import re

def extract_paper_titles(text):
    patterns = [
        r"'([^']+)'.*?'([^']+)'.*?which one.*?(?:published|authors)|which one.*?(?:published|authors).*?'([^']+)'.*?'([^']+)'",
        r"(?:\bpaper\s+|\bauthors\s+of\s+)'([^']+)'"
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            # Filter out None values from groups
            return [g for g in match.groups() if g is not None]

    return None

# Test
print(extract_paper_titles("Which one was published first, 'Fast Spectral Clustering with efficient large graph construction' or 'Lifetime Inference for Highly Reliable Products Based on Skew-Normal Accelerated Destructive Degradation Test Model'?"))


['Fast Spectral Clustering with efficient large graph construction', 'Lifetime Inference for Highly Reliable Products Based on Skew-Normal Accelerated Destructive Degradation Test Model']


In [3]:
import requests

def get_paper_url(title_or_doi: str):
    # Check if input looks like a DOI
    if '/' in title_or_doi and '.' in title_or_doi.split('/')[0]:
        doi = title_or_doi.strip()
    else:
        # Search CrossRef by title
        params = {"query.title": title_or_doi, "rows": 1}
        response = requests.get("https://api.crossref.org/works", params=params)
        if response.status_code != 200:
            return f"Error fetching data: {response.status_code}"
        items = response.json()["message"]["items"]
        if not items:
            return "No results found for the given title."
        doi = items[0]["DOI"]

    # Build DOI link
    return f"https://doi.org/{doi}"

# Examples:
print(get_paper_url("Minimally Supervised Learning of Affective Events Using Discourse Relations"))


https://doi.org/10.18653/v1/d19-1581


In [4]:
import requests

def get_pdf_url(title: str, email="amel.gader@uni-passau.de"):
    # Ensure DOI format
    doi = get_paper_url(title)
    doi = doi.strip()

    # Call Unpaywall API
    api_url = f"https://api.unpaywall.org/v2/{doi}"
    params = {"email": email}  # Required for Unpaywall
    response = requests.get(api_url, params=params)

    if response.status_code != 200:
        return f"Error: {response.status_code} - {response.text}"

    data = response.json()

    # Check for open access fulltext
    oa_location = data.get("best_oa_location")
    if oa_location and oa_location.get("url_for_pdf"):
        return oa_location["url_for_pdf"]
    else:
        return None



In [6]:
# Example usage:
doi = "https://doi.org/10.18653/v1/d19-1581"  # Replace with your DOI
print(get_pdf_url(doi))


None


In [7]:
import requests

import os
import requests

def download_pdf_to_repo(doi: str, repo_pdf_dir: str = "pdfs"):
    os.makedirs(repo_pdf_dir, exist_ok=True)
    
    pdf_url = get_pdf_url(doi)
    print(f"PDF URL: {pdf_url}")
    
    if not pdf_url:
        print("No open-access PDF found for this DOI.")
        return
    doi = doi.strip().replace("https://doi.org/", "")
    filename = doi.replace("/", "_") + ".pdf"
    output_path = os.path.join(repo_pdf_dir, filename)

    if os.path.exists(output_path):
        print(f"PDF already exists at {output_path}")
        return

    response = requests.get(pdf_url)
    if response.status_code == 200 and response.headers.get("Content-Type", "").startswith("application/pdf"):
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"PDF downloaded and saved to {output_path}")
    else:
        print("Failed to download PDF or URL is not a PDF.")




In [8]:
download_pdf_to_repo(doi=doi)

PDF URL: https://www.protocols.io/view/doi-test-cgwftxbn.pdf
PDF downloaded and saved to pdfs/10.18653_v1_d19-1581.pdf


In [1]:
from datasets import load_dataset

# This fetches the full version, not the dummy one
dataset = load_dataset("allenai/qasper")

# Convert to pandas
df = dataset['train'].to_pandas()

# Check number of records
print(len(df))


/root/kg-qa/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 416/416 [00:00<00:00, 1251.83 examples/s]


888


In [9]:
for title in df['title'].unique():
    download_pdf("/papers", title)
    

Downloaded to /papers
Downloaded to /papers
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
Downloaded to /papers
No open-access PDF found for this DOI.
Downloaded to /papers
Downloaded to /papers
Failed to download PDF or URL is not a PDF.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
Failed to download PDF or URL is not a PDF.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
Downloaded to /papers
Downloaded to /papers
Downloaded to /papers
Downloaded to /papers
Downloaded to /papers
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for this DOI.
No open-access PDF found for 

InvalidSchema: No connection adapters were found for 'Error: 404 - <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'